In [30]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import os
import collections
import time
import math
import pickle
import re
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model

In [31]:
tf.__version__

'1.15.4'

In [32]:
train_labels = open("SR-ARE-train/names_labels.txt","r")
content = train_labels.read()
#print(content)

In [33]:
content_list = re.split(",|\n",content)
#print(content_list)
#print(len(content_list))

print(content_list.count("0"),content_list.count("1"))

6069 1098


In [34]:
drug_dict = {content_list[2*i]:float(content_list[2*i+1]) for i in range(len(content_list)//2)}
#print(drug_dict)
#print(len(drug_dict))

In [35]:
train_smiles = open("SR-ARE-train/names_smiles.txt","r")
dtent = train_smiles.read()
#print(dtent)

In [36]:
dtent_list = re.split(",|\n",dtent)
drug_name_dict = {dtent_list[2*i]:dtent_list[2*i+1] for i in range(len(dtent_list)//2)}
#print(drug_name_dict)
#print(len(drug_name_dict))

In [37]:
train_one_hot = open("SR-ARE-train/names_onehots.pickle","rb")
rtent = pickle.load(train_one_hot)
#print(rtent)

In [38]:
print(type(rtent))
print(type(rtent["onehots"]))
print(rtent["onehots"].shape)


<class 'dict'>
<class 'numpy.ndarray'>
(7167, 70, 325)


In [39]:
print(type(rtent["names"]))

<class 'list'>


In [40]:
x_train = rtent["onehots"]
label = rtent["names"]
zlabel = [drug_dict[label[i]] for i in range(len(label))]
#print(len(zlabel))
y_train = np.array(zlabel,float)
print(x_train.shape)
print(y_train.shape)


(7167, 70, 325)
(7167,)


In [41]:
correct = y_train >= 0.5
print(correct)
true_label = correct[correct].astype(int)
print(true_label)
print(len(true_label))
true_find = x_train[correct]
print(len(true_find))

incorrect = y_train <= 0.5
false_label = incorrect[incorrect].astype(int)
false_find = x_train[incorrect]
print(len(false_label))

[False  True False ... False False  True]
[1 1 1 ... 1 1 1]
1098
1098
6069


In [42]:
weight_for_zero = 1.0 / len(false_label)
weight_for_one = 1.0 / len(true_label)
print(weight_for_zero,weight_for_one)

0.00016477179106936892 0.0009107468123861566


In [43]:
drug_height = x_train.shape[1]
print(drug_height)
drug_width = x_train.shape[2]
print(drug_width)
drug_shape = (drug_height , drug_width)
print(drug_shape)
drug_full_shape = (drug_height , drug_width,1)
print(drug_full_shape)
no_class = 1
no_channel = 1

70
325
(70, 325)
(70, 325, 1)


In [15]:
def find_formula(dictlist,namelabel,y):
    for i in range(9):
        print(namelabel[i],dictlist[namelabel[i]],y[i])

In [14]:
find_formula(drug_name_dict,label[0:9],y_train[0:9])

NCGC00254346-01 CC(C)=CCC\C(C)=C/CO 0.0
NCGC00254668-01 FC1=CC=CC(F)=C1C(=O)NC(=O)NC2=C(F)C=C(OC3=C(Cl)C=C(C=C3)C(F)(F)F)C=C2 1.0
NCGC00015646-05 Cl.CC1=CNN=C1 0.0
NCGC00255698-01 CC1(C)[C@H](C=C(Cl)Cl)[C@@H]1C(=O)O[C@@H](C#N)C2=CC=CC(OC3=CC=CC=C3)=C2 0.0
NCGC00160628-03 C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](OC3=C(OC4=C(C3=O)C(O)=CC(O)=C4)C5=CC(O)=C(O)C=C5)[C@H](O)[C@@H](O)[C@@H]2O)[C@H](O)[C@H](O)[C@H]1O 0.0
NCGC00255567-01 CCCCCCOC1=C(C=CC=C1)C(O)=O 0.0
NCGC00262943-01 OCCNCCO.CC(OC(=O)C1(C)CCC(C(O)=O)C1(C)C)C2=CC=C(C)C=C2 0.0
NCGC00256129-01 CCCCOC(=O)CCCCCCCC(=O)OCCCC 0.0
NCGC00016666-06 Br.COC1=C2CN(C)CCC2=CC3=C1OCO3 0.0


In [16]:
def find_example_error(pred,dictlist,namelabel,y):
    incorrect = (pred != y)
    namelabel = namelabel[incorrect]
    pred = pred[incorrect]
    find_formula(dictlist,namelabel[0:9],y[0:9])

In [18]:
model = Sequential()
model.add(InputLayer(input_shape=(drug_height,drug_width,)))

model.add(Reshape(drug_full_shape))

model.add(Conv2D(kernel_size=(1,16),strides=1,filters=8,padding='same',activation='relu',name="conv1"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,2), strides=(1,2),padding='same'))

model.add(Conv2D(kernel_size=(1,16),strides=1,filters=16,padding='same',activation='relu',name="conv2"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,2), strides=(1,2),padding='same'))

model.add(Conv2D(kernel_size=(1,16),strides=1,filters=32,padding='same',activation='relu',name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,2), strides=(1,2),padding='same'))

model.add(Flatten())

model.add(Dense(140,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(no_class,activation="sigmoid"))



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [19]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 70, 325, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 70, 325, 8)        136       
_________________________________________________________________
batch_normalization (BatchNo (None, 70, 325, 8)        32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 70, 163, 8)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 70, 163, 16)       2064      
_________________________________________________________________
batch_normalization_1 (Batch (None, 70, 163, 16)       64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 70, 82, 16)        0

In [20]:
optimizer = Adam(lr=1e-4)

In [21]:
metrics = [
    keras.metrics.BinaryAccuracy(name="accuracy"),
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

In [26]:
class_weight = {0: weight_for_zero, 1: weight_for_one}

In [23]:
model.compile(optimizer=optimizer,loss="binary_crossentropy", metrics=metrics)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [97]:
model.fit(x=x_train, y=y_train, epochs=1, batch_size=128,class_weight=class_weight)

Train on 7167 samples
7167/7167 [==============================] - 233s 33ms/sample - loss: 1.5290e-05 - accuracy: 0.9833 - fn: 13.0000 - fp: 107.0000 - tn: 5962.0000 - tp: 1085.0000 - precision: 0.9102 - recall: 0.9882


In [ ]:
#30 epochs

In [46]:
test_labels = open("SR-ARE-test/names_labels.txt","r")
ttcontent = test_labels.read()

In [47]:
ttcontent_list = re.split(",|\n",ttcontent)
#print(ttcontent_list)
#print(len(ttcontent_list))
print(ttcontent_list.count("0"),ttcontent_list.count("1"))

186 48


In [48]:
ttdrug_dict = {ttcontent_list[2*i]:float(ttcontent_list[2*i+1]) for i in range(len(ttcontent_list)//2)}
#print(drug_dict)
#print(len(drug_dict))

In [49]:
test_smiles = open("SR-ARE-test/names_smiles.txt","r")
ttdtent = test_smiles.read()
#print(ttdtent)

In [50]:
ttdtent_list = re.split(",|\n",ttdtent)
ttdrug_name_dict = {ttdtent_list[2*i]:ttdtent_list[2*i+1] for i in range(len(ttdtent_list)//2)}
#print(ttdrug_name_dict)
print(len(ttdrug_name_dict))

234


In [51]:
test_one_hot = open("SR-ARE-test/names_onehots.pickle","rb")
ttrtent = pickle.load(test_one_hot)
#print(rtent)

In [52]:
x_test = ttrtent["onehots"]
ttlabel = ttrtent["names"]
ttzlabel = [ttdrug_dict[ttlabel[i]] for i in range(len(ttlabel))]
#print(len(zlabel))
y_test = np.array(ttzlabel,float)

In [98]:
result = model.evaluate(x=x_test,y=y_test)

234/234 [==============================] - 1s 5ms/sample - loss: 0.9415 - accuracy: 0.7479 - fn: 39.0000 - fp: 20.0000 - tn: 166.0000 - tp: 9.0000 - precision: 0.3103 - recall: 0.1875


In [63]:
model.fit(x=x_test, y=y_test, epochs=2, batch_size=64)

Train on 234 samples
Epoch 1/2
234/234 [==============================] - 6s 27ms/sample - loss: 0.1561 - accuracy: 0.9444 - fn: 7.0000 - fp: 6.0000 - tn: 180.0000 - tp: 41.0000 - precision: 0.8723 - recall: 0.8542
Epoch 2/2
234/234 [==============================] - 7s 28ms/sample - loss: 0.0882 - accuracy: 0.9701 - fn: 7.0000 - fp: 0.0000e+00 - tn: 186.0000 - tp: 41.0000 - precision: 1.0000 - recall: 0.8542


In [81]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

loss 0.6891390533528776
acc 0.7905983


In [99]:
#save model
path_model = 'model/model2-1.keras'
model.save(path_model)

In [66]:
path_model = 'model/model2-1.keras'
model = tf.keras.models.load_model(path_model)

In [33]:
pred_test = model.predict(x=x_test)
print(pred_test)

[[0.4739284 ]
 [0.47804084]
 [0.46564117]
 [0.46704525]
 [0.47187254]
 [0.47058776]
 [0.465124  ]
 [0.46949452]
 [0.4864085 ]
 [0.4693296 ]
 [0.4780901 ]
 [0.46737242]
 [0.48747426]
 [0.49174497]
 [0.47168413]
 [0.4722297 ]
 [0.47147113]
 [0.4722702 ]
 [0.4645362 ]
 [0.46777347]
 [0.4670035 ]
 [0.46454284]
 [0.47184926]
 [0.4678263 ]
 [0.47725335]
 [0.46419954]
 [0.48286912]
 [0.47849268]
 [0.4753865 ]
 [0.47663707]
 [0.46531707]
 [0.47669727]
 [0.47997656]
 [0.46524394]
 [0.47358137]
 [0.4731608 ]
 [0.47263566]
 [0.4663907 ]
 [0.4762288 ]
 [0.4689682 ]
 [0.47034287]
 [0.4749545 ]
 [0.47354317]
 [0.4734465 ]
 [0.47255018]
 [0.47863886]
 [0.46935716]
 [0.46632063]
 [0.47340816]
 [0.47573927]
 [0.47246367]
 [0.47454286]
 [0.46573448]
 [0.4679682 ]
 [0.46832836]
 [0.48033926]
 [0.47279117]
 [0.47572324]
 [0.4651072 ]
 [0.47401324]
 [0.46596706]
 [0.46430013]
 [0.47487006]
 [0.47323728]
 [0.4734092 ]
 [0.47130877]
 [0.46437982]
 [0.46599114]
 [0.47081602]
 [0.46252587]
 [0.47257516]
 [0.46

In [34]:
checking = pred_test >= 0.5
print(checking)

[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 

In [35]:
answer = checking.astype(int)
print(answer)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [93]:
f = open("labels.txt", "w")
for i in range(len(answer)):
    f.write(str(answer[i][0]) + "\n")
f.close()